In [68]:
import pandas as pd           
import praw                   
import re                     
import datetime as dt
import seaborn as sns
import requests
import json
## acknowledgements
'''
https://stackoverflow.com/questions/48358837/pulling-reddit-comments-using-python-praw-and-creating-a-dataframe-with-the-resu
https://www.reddit.com/r/redditdev/comments/2e2q2l/praw_downvote_count_always_zero/
https://towardsdatascience.com/an-easy-tutorial-about-sentiment-analysis-with-deep-learning-and-keras-2bf52b9cba91
'''

'\nhttps://stackoverflow.com/questions/48358837/pulling-reddit-comments-using-python-praw-and-creating-a-dataframe-with-the-resu\nhttps://www.reddit.com/r/redditdev/comments/2e2q2l/praw_downvote_count_always_zero/\nhttps://towardsdatascience.com/an-easy-tutorial-about-sentiment-analysis-with-deep-learning-and-keras-2bf52b9cba91\n'

In [4]:
with open("API.env") as file:
    exec(file.read())

reddit = praw.Reddit(
  client_id = client_id,
  client_secret = client_secret,
  user_agent = user_agent
)

Version 7.0.0 of praw is outdated. Version 7.2.0 was released Wednesday February 24, 2021.


In [43]:
'''
Some helper functions for the reddit API.
'''

def extract_num_rewards(awardings_data):
    return sum( x["count"] for x in awardings_data)

def extract_data(submission, comments = True):
    postlist = []

    # extracts top level comments
    if comments:
        submission.comments.replace_more(limit=0)
        for comment in submission.comments: 
            post = {} # put this here
            post['author'] = comment.author
            post['body'] = comment.body
            post['awards'] = comment.all_awardings
            post['score'] = comment.score
            post['parent_id'] = comment.parent_id
            post['id'] = comment.id

            postlist.append(post)

    content = {
    "title" : submission.title,
    "self" : submission.is_self,
    "text" : submission.selftext,
    "comments" : postlist,
    "author" : submission.author,
    "name" : submission.name,
    "upvote_ratio" : submission.upvote_ratio,
    "ups" : submission.score, #this is the same as submission.ups,
    "downs" : None,
    "awarders" : submission.awarders, 
    "awards" : submission.all_awardings,
    "total_awards" : None,
    "url" : submission.url # Only relevent if not a self post
    }
    
    content["total_awards"] = extract_num_rewards(content["awards"])
    content["downs"] = (1 - content["upvote_ratio"]) * content["ups"] / content["upvote_ratio"]
    return content

In [44]:
'''
Sample num_samples random submissions, and get the top num_samples submissions, and put them into dataframes.
'''

def random_sample(num_samples, subreddit):
    sample = []
    for i in range(num_samples):
        submission = reddit.subreddit(subreddit).random() 
        sample.append(extract_data(submission))
    return(pd.DataFrame(sample))

def sample(source):
    submissions = []
    for submission in source:
        submissions.append(extract_data(submission))
    print(f"Got {len(submissions)} submissions. (This can be less than num_samples.)")
    return(pd.DataFrame(submissions))

def top_sample(num_samples, subreddit):
    return sample(reddit.subreddit(subreddit).top(limit=num_samples) )

def rising_sample(num_samples, subreddit):
    return sample(reddit.subreddit(subreddit).rising(limit=num_samples))

def controversial_sample(num_samples, subreddit):
    return sample(reddit.subreddit(subreddit).controversial(limit=num_samples) )


num_samples = 10
subreddit ='wallstreetbets'

random_wsb = random_sample(num_samples, subreddit)
top_wsb = top_sample(num_samples,subreddit)
rising_wsb = rising_sample(num_samples, subreddit)
controversial_wsb = controversial_sample(num_samples, subreddit)
# other commands here: https://praw.readthedocs.io/en/latest/code_overview/models/subreddit.html#praw.models.Subreddit.rising
# NB: The subreddit stream option seems useful.
# NB: There is also rising_random



Got 10 submissions. (This can be less than num_samples.)
Got 10 submissions. (This can be less than num_samples.)
Got 10 submissions. (This can be less than num_samples.)


In [45]:
random_wsb.to_pickle("random_wsb.pkl")
top_wsb.to_pickle("top_wsb.pkl")
rising_wsb.to_pickle("rising_wsb.pkl")
controversial_wsb.to_pickle("controversial_wsb.pkl")

In [111]:
'''
Code for getting all submissions between certain date time

https://www.reddit.com/r/redditdev/comments/5gfvik/praw_getting_all_submissions_for_the_past_two/

for submission in reddit.subreddit('subreddit').submissions(past_timestamp, now_timestamp):
    print(submission.created_utc)
    
This no longer works, see here: https://stackoverflow.com/questions/53988619/praw-6-get-all-submission-of-a-subreddit

Example of using pushshift: https://github.com/Watchful1/Sketchpad/blob/master/postDownloader.py

'''

past = dt.datetime(2021, 5, 5)
past_timestamp = int(past.replace(tzinfo=dt.timezone.utc).timestamp())

now = dt.datetime.now()
now_timestamp = int(now.replace(tzinfo=dt.timezone.utc).timestamp())
now_timestamp

url = f"https://api.pushshift.io/reddit/submission/search/?after={past_timestamp}&before={now_timestamp}&sort_type=score&sort=desc&subreddit=wallstreetbets&limit=1000"

data = requests.get(url)
data_json = data.json()
wsb_df = pd.DataFrame(data_json['data'])
wsb_cleaned = wsb_df[['title', 'created_utc', 'author_fullname', 'is_self', 'score', 'upvote_ratio', 'total_awards_received', 'selftext']]

In [112]:
wsb_cleaned

,title,created_utc,author_fullname,is_self,score,upvote_ratio,total_awards_received,selftext
0,Road to a million. PT - 4.20,1620188932,t2_cgv2s,False,2,1.00,0,
1,AMC to the fucking moon!,1620188150,t2_amfmcir4,True,2,1.00,0,[removed]
2,$ADVM,1620192122,t2_996goyhn,True,2,1.00,0,thoughts on ADVM
3,who is holding until $1?,1620247136,t2_833jvfk5,False,2,1.00,0,
4,"Quick, the mods are on vacation today! Finally...",1620291886,t2_wjufi,False,2,0.75,0,
...,...,...,...,...,...,...,...,...
95,GME - 2021 Annual Meeting - VOT3,1620213700,t2_41wicud8,True,1,1.00,0,[removed]
96,Getting started,1620215264,t2_9kb8uvo4,True,1,1.00,0,[removed]
97,$AHT calls gonna print money tm,1620173223,t2_1zn5jf9,False,1,1.00,0,
98,TLRY and tobacco giant Philip morris,1620172910,t2_7sd456rk,True,1,1.00,0,Only real danger to tlry are big tobacco compa...


In [13]:
# traffic = reddit.subreddit(subreddit).traffic() is not available to us, sadly.

,title,self,text,comments,author,name,upvote_ratio,ups,downs,awarders,awards,total_awards,url
0,"RKT Megathread for March 2nd, 2021",True,"RKT was highly discussed yesterday, so please ...","[{'author': OPINION_IS_UNPOPULAR, 'body': '[GM...",OPINION_IS_UNPOPULAR,t3_lw6hoh,0.54,1550,1320.370370,[],"[{'giver_coin_reward': 0, 'subreddit_id': None...",106,https://www.reddit.com/r/wallstreetbets/commen...
1,"Daily Discussion Thread for April 20, 2021",True,Your daily trading discussion thread. Please k...,"[{'author': theycallmeryan, 'body': 'Happy 4/2...",OPINION_IS_UNPOPULAR,t3_mumzmy,0.51,53,50.921569,[],"[{'giver_coin_reward': None, 'subreddit_id': N...",28,https://www.reddit.com/r/wallstreetbets/commen...
2,FUCK SILVER. Here's the move AFTER the squeeze...,True,EDIT: Downvote all you want SLV shills. Mariju...,"[{'author': InfestedSinner, 'body': 'Sundials ...",OverwatchNWeed,t3_l9laqm,0.51,36,34.588235,[],"[{'giver_coin_reward': 0, 'subreddit_id': None...",9,https://www.reddit.com/r/wallstreetbets/commen...
3,$NOK still trying to go to the moon??? 🚀🚀🚀🚀🚀🚀🚀...,True,,"[{'author': HugeGuyLL, 'body': 'Dear Autists, ...",dodr6773,t3_l6d4qy,0.49,0,0.000000,[],"[{'giver_coin_reward': 0, 'subreddit_id': None...",5,https://www.reddit.com/r/wallstreetbets/commen...
4,Stop letting random idiots post their plays an...,True,Why are letting any random person of the 8mil ...,"[{'author': BullShitting24-7, 'body': 'Ban', '...",SweetGus,t3_lbfoj4,0.47,0,0.000000,[],"[{'giver_coin_reward': None, 'subreddit_id': '...",5,https://www.reddit.com/r/wallstreetbets/commen...
5,Fidelity Dropped GME Stock from 9.3M to 87 Sha...,True,So all those calculations about institutional ...,"[{'author': dualshock7, 'body': 'Person didn't...",broneilbro,t3_lh173n,0.52,37,34.153846,[],"[{'giver_coin_reward': None, 'subreddit_id': N...",5,https://www.reddit.com/r/wallstreetbets/commen...
6,The DD on SLV - Updated 1/31 - clarity on what...,True,Edit: it appears someone may have created bots...,"[{'author': lenti_virus, 'body': 'Silver is a ...",TheHappyHawaiian,t3_l9tzkv,0.50,86,86.000000,[],"[{'giver_coin_reward': 0, 'subreddit_id': None...",28,https://www.reddit.com/r/wallstreetbets/commen...
7,SILVER SURFERS 🏄‍♂️ RISE UP,False,,"[{'author': CallsOnAlcoholism, 'body': 'Amazin...",TUNISIANP,t3_l8yph2,0.49,0,0.000000,[],"[{'giver_coin_reward': 0, 'subreddit_id': None...",10,https://i.redd.it/ucmqhooc0ke61.jpg
8,RobinHood Insider Alleges White House Pressure...,False,,"[{'author': SweatyControles, 'body': 'How woul...",iphone5000,t3_l79jy6,0.51,32,30.745098,[],[],0,https://www.hiddenamericans.com/robinhood-insi...
9,GME rational consideration from a professional...,True,I know it feels like you guys have discovered ...,"[{'author': supermina97, 'body': 'ARE YOU HEAR...",vegaseller,t3_l7kgnc,0.48,0,0.000000,[],"[{'giver_coin_reward': None, 'subreddit_id': N...",2,https://www.reddit.com/r/wallstreetbets/commen...


In [36]:
reddit.subreddit(subreddit).top(limit=num_samples)

AttributeError: 'ListingGenerator' object has no attribute 'name'